In [2]:
!pip install -U awswrangler


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
# imports and setup
from time import sleep
import time
import boto3
import sagemaker
from sagemaker.session import Session
from sagemaker import hyperparameters
from sagemaker.tuner import ContinuousParameter, IntegerParameter, HyperparameterTuner
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep, TuningStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker import Model
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.workflow.model_step import ModelStep
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    Processor,
    ScriptProcessor,
)
from sagemaker.inputs import TrainingInput
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join


sess = sagemaker.Session()
bucket = sess.default_bucket()
#role = sagemaker.get_execution_role()
region = boto3.Session().region_name

#s3_client = boto3.client("s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
# Create a Pipeline Session
pipeline_session = PipelineSession()

In [5]:
# set some basic pipeline parameters stored
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
mse_threshold = ParameterFloat(name="MseThreshold", default_value=6.0)

In [6]:
step_fail = FailStep(
    name="SmartGridMSEFail",
    error_message=Join(on=" ", values=["Execution failed due to MSE >", mse_threshold]),
)

In [7]:
# import custom library
import pipe_tools

In [8]:
# set all data filters
def train_filter(partitions):
    return (int(partitions["year"]) == 2012) 

def val_filter(partitions):
    return (int(partitions["year"]) == 2013) and (int(partitions["month"]) <= 2) 

def batch_filter(partitions):
    return (int(partitions["year"]) == 2012) and (int(partitions["month"]) <= 4) and (int(partitions["month"]) > 2)

In [9]:
# get all data from filters
pq_data = pipe_tools.get_pqdata(bucket, train_filter, val_filter, batch_filter)

/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)
/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)
/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)
/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)


In [10]:
# demonstrate data
pq_data['val']

,kWh,weekday,hour,temperature,Holiday
timestamp,,,,,
2013-01-01 00:00:00,0.250960,1.0,0.0,7.01,5.0
2013-01-01 01:00:00,0.211900,1.0,1.0,7.49,5.0
2013-01-01 02:00:00,0.177964,1.0,2.0,7.16,5.0
2013-01-01 03:00:00,0.149907,1.0,3.0,7.04,5.0
2013-01-01 04:00:00,0.133761,1.0,4.0,6.29,5.0
...,...,...,...,...,...
2013-02-28 19:00:00,0.400435,3.0,19.0,4.17,-1.0
2013-02-28 20:00:00,0.394198,3.0,20.0,4.36,-1.0
2013-02-28 21:00:00,0.376216,3.0,21.0,3.90,-1.0


In [11]:
# set up feature store session
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

# record identifier and event time feature names
record_identifier_feature_name = "timestamp"
event_time_feature_name = "EventTime"

pq_data2 = pq_data.copy()

f_groups = pipe_tools.setup_feature_groups(pq_data2, feature_store_session, event_time_feature_name)

In [12]:
# set the  feature store bucket
feat_s3_bucket_name = feature_store_session.default_bucket()

In [13]:
%%time
# train-val-batch data upload
prefix = "lgbm-fs-final"

dataset_uris = {}
for k in pq_data.keys():
    uri_prefix = f"s3://{feat_s3_bucket_name}/{prefix}/{k}_input/"
    dataset_uris[k] = uri_prefix
    pq_data[k].to_parquet(f"{k}_dataset.parquet")
    sess.upload_data(
        f"{k}_dataset.parquet", bucket=bucket, key_prefix=prefix + f"/{k}_input"
    )

dataset_uris

CPU times: user 39.9 ms, sys: 1.84 ms, total: 41.7 ms
Wall time: 379 ms


{'train': 's3://sagemaker-us-east-1-645257835735/lgbm-fs-final/train_input/',
 'val': 's3://sagemaker-us-east-1-645257835735/lgbm-fs-final/val_input/',
 'batch': 's3://sagemaker-us-east-1-645257835735/lgbm-fs-final/batch_input/'}

In [14]:
# get the execution role
from sagemaker import get_execution_role
role = get_execution_role()

In [15]:
# create feature stores
for f in f_groups:
    f.create(
        s3_uri=f"s3://{feat_s3_bucket_name}/{prefix}",
        record_identifier_name=record_identifier_feature_name,
        event_time_feature_name=event_time_feature_name,
        role_arn=role,
        enable_online_store=True,
    )

# wait for completion
for f in f_groups:
    pipe_tools.wait_for_feature_group_creation_complete(feature_group=f)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup train-feature-group-04-00-49-03 successfully created.
Waiting for Feature Group Creation
FeatureGroup val-feature-group-04-00-49-03 successfully created.
FeatureGroup batch-feature-group-04-00-49-03 successfully created.


In [16]:
# set up inputs for training
train_data = TrainingInput(
    dataset_uris['train'],
    distribution="FullyReplicated",
    content_type="application/x-parquet",
    s3_data_type="S3Prefix",
)
validation_data = TrainingInput(
    dataset_uris['val'],
    distribution="FullyReplicated",
    content_type="application/x-parquet",
    s3_data_type="S3Prefix",
)

data_channels = {"train": train_data, 'validation': validation_data}

# set up validation for eval script
eval_data = ParameterString(
    name="InputDataUrl",
    default_value=dataset_uris['val'],
)

In [17]:
# set model variables
train_model_id = "lightgbm-regression-model"
train_model_version = "2.1.0"
train_scope = "training"
training_instance_type = "ml.m5.xlarge"

# get model URIs
model_uri = pipe_tools.setup_uris(train_model_id, train_model_version, train_scope, training_instance_type)

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)
hyperparameters["num_boost_round"] = "500"  
print(hyperparameters)

{'num_boost_round': '500', 'early_stopping_rounds': '30', 'metric': 'auto', 'learning_rate': '0.009', 'num_leaves': '67', 'feature_fraction': '0.74', 'bagging_fraction': '0.53', 'bagging_freq': '5', 'max_depth': '11', 'min_data_in_leaf': '26', 'max_delta_step': '0.0', 'lambda_l1': '0.0', 'lambda_l2': '0.0', 'boosting': 'gbdt', 'min_gain_to_split': '0.0', 'tree_learner': 'serial', 'feature_fraction_bynode': '1.0', 'is_unbalance': 'False', 'max_bin': '255', 'tweedie_variance_power': '1.5', 'num_threads': '0', 'verbosity': '1', 'use_dask': 'False'}


In [18]:
# use the parameter tuner
use_amt = True

In [19]:
# set tuner hyperparameter ranges
if train_model_id == "lightgbm-regression-model":
    hyperparameter_ranges = {
        "learning_rate": ContinuousParameter(1e-4, 1, scaling_type="Logarithmic"),
        "num_boost_round": IntegerParameter(2, 30),
        "early_stopping_rounds": IntegerParameter(2, 30),
        "num_leaves": IntegerParameter(10, 50),
        "feature_fraction": ContinuousParameter(0, 1),
        "bagging_fraction": ContinuousParameter(0, 1),
        "bagging_freq": IntegerParameter(1, 10),
        "max_depth": IntegerParameter(5, 30),
        "min_data_in_leaf": IntegerParameter(5, 50),
    }

In [20]:
# create model estimator
from sagemaker.estimator import Estimator
prefix_output = "LGBMTrain"
training_output_path = f"s3://{feat_s3_bucket_name}/{prefix_output}"
LGBM_Estimator = Estimator(
    role=role,
    image_uri=model_uri['image'],
    source_dir=model_uri['source'],
    model_uri=model_uri['model'],
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=training_output_path,
    sagemaker_session = pipeline_session
)

In [21]:
# set the fit parameters in the pipeline
from sagemaker.utils import name_from_base

training_job_name = name_from_base(f"SmartGrid-{train_model_id}-training")

if use_amt:
    tuner = HyperparameterTuner(
        LGBM_Estimator,
        "rmse",
        hyperparameter_ranges,
        [{"Name": "rmse", "Regex": "rmse: ([0-9\\.]+)"}],
        max_jobs=10,
        max_parallel_jobs=4,
        objective_type="Minimize",
        base_tuning_job_name=training_job_name,
    )

    hpo_args = tuner.fit(inputs=data_channels)
    
    step_tuning = TuningStep(
        name="HPTuning",
        step_args=hpo_args,
    )

else:
    # Launch a SageMaker Training job by passing s3 path of the training data
    LGBM_Estimator.fit(
        inputs=data_channels, logs=True, job_name=training_job_name
    )

/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [22]:
inference_instance_type = "ml.m5.large"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=train_model_id,
    model_version=train_model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope="inference"
)

In [23]:
# model creation section of the pipeline
best_model = Model(
    image_uri=deploy_image_uri,
    model_data=step_tuning.get_top_model_s3_uri(
        top_k=0, s3_bucket=feat_s3_bucket_name, prefix=prefix_output
    ),
    sagemaker_session=pipeline_session,
    role=role,
)

step_create_first = ModelStep(
    name="CreateBestModel",
    step_args=best_model.create(instance_type=inference_instance_type),
)

second_best_model = Model(
    image_uri=deploy_image_uri,
    model_data=step_tuning.get_top_model_s3_uri(
        top_k=1, s3_bucket=feat_s3_bucket_name, prefix=prefix_output
    ),
    sagemaker_session=pipeline_session,
    role=role,
)

step_create_second = ModelStep(
    name="CreateSecondBestModel",
    step_args=second_best_model.create(instance_type=inference_instance_type),
)

In [24]:
%%writefile code/evaluate.py
# evaluation script
"""Evaluation script for measuring mean squared error."""

import subprocess
import sys

# Install packages that aren't in the runtime
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    
install('lightgbm')
install('pyarrow')

import json
import logging
import pathlib
import tarfile
import joblib 
import lightgbm

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())


if __name__ == "__main__":
    logger.debug("Starting evaluation.")
    model_path = "/opt/ml/processing/model/model.tar.gz"
    t = tarfile.open(model_path, 'r:gz')
    t.extractall()

    logger.debug("Loading lgbm model.")
    model = joblib.load("model.pkl")

    logger.debug("Reading test data.")
    test_path = "/opt/ml/processing/val/val_dataset.parquet"
    df = pd.read_parquet(test_path)

    logger.debug("Extracting y column and preparing test data.")
    y_test = df["kWh"]
    df.drop(columns=["kWh"], inplace=True)
    

    logger.info("Performing predictions against test data.")
    predictions = model.predict(df)

    logger.debug("Calculating mean squared error.")
    mse = mean_squared_error(y_test, predictions)
    std = np.std(y_test - predictions)
    report_dict = {
        "regression_metrics": {
            "mse": {"value": mse, "standard_deviation": std},
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    logger.info("Writing out evaluation report with mse: %f", mse)
    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting code/evaluate.py


In [25]:
# A ProcessingStep is used to evaluate the performance of a selected model from the HPO step. In this case, the top performing model
# is evaluated. Based on the results of the evaluation, the model is registered into the Model Registry using a ConditionStep.

script_prefix= "SmartGrid/script-tuning-step-eval"

script_eval = ScriptProcessor(
    image_uri=deploy_image_uri,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name=script_prefix,
    sagemaker_session=pipeline_session,
    role=role,
)

evaluation_report = PropertyFile(
    name="BestTuningModelEvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

processor_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_tuning.get_top_model_s3_uri(
                top_k=0, s3_bucket=feat_s3_bucket_name, prefix=prefix_output
            ),
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=dataset_uris['val'],
            destination="/opt/ml/processing/val",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="code/evaluate.py",
)

# This can be extended to evaluate multiple models from the HPO step
step_eval = ProcessingStep(
    name="EvaluateTopModel",
    step_args=processor_args,
    property_files=[evaluation_report],
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

In [26]:
# Register the model in the Model Registry
# Multiple models can be registered into the Model Registry using multiple ModelSteps. These models can either be added to the
# same model package group as different versions within the group or the models can be added to different model package groups.

model_package_group_name = "tuning-job-model-packages"

register_args = best_model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
)


step_register_best = ModelStep(name="RegisterBesLGBTModel", step_args=register_args)

In [27]:
# Conditional evaluation
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.mse.value",
    ),
    right=mse_threshold,
)

# conditional steps
step_cond = ConditionStep(
    name="CheckMSEEvaluation",
    conditions=[cond_lte],
    if_steps=[step_register_best],
    else_steps=[step_fail],
)

In [28]:
# set up pipeline and all steps
pipeline = Pipeline(
    name="tuning-step-pipeline",
    parameters=[
        training_instance_type,
        model_approval_status,
        mse_threshold,
    ],
    steps=[
        step_tuning,
        step_create_first,
        step_create_second,
        step_eval,
        step_cond,
    ],
    sagemaker_session=pipeline_session,
)

In [29]:
import json
# Show the pipeline definitions
definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'MseThreshold', 'Type': 'Float', 'DefaultValue': 6.0}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'HPTuning',
   'Type': 'Tuning',
   'Arguments': {'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian',
     'ResourceLimits': {'MaxNumberOfTrainingJobs': 10,
      'MaxParallelTrainingJobs': 4},
     'TrainingJobEarlyStoppingType': 'Off',
     'HyperParameterTuningJobObjective': {'Type': 'Minimize',
      'MetricName': 'rmse'},
     'ParameterRanges': {'ContinuousParameterRanges': [{'Name': 'learning_rate',
        'MinValue': '0.0001',
        'MaxValue': '1',
        'ScalingType': 'Logarithmic'},
       {'Name': 'feature_fraction',
        'MinValue': '0',
        'MaxValue': '1',
        'ScalingTyp

In [30]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:645257835735:pipeline/tuning-step-pipeline',
 'ResponseMetadata': {'RequestId': '968282f7-2a62-4853-ade6-0725c1f83dd5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '968282f7-2a62-4853-ade6-0725c1f83dd5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Mon, 04 Mar 2024 00:49:42 GMT'},
  'RetryAttempts': 0}}

In [31]:
# start pipeline
execution = pipeline.start()

In [32]:
# show execution
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:645257835735:pipeline/tuning-step-pipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:645257835735:pipeline/tuning-step-pipeline/execution/xq4mq73mqxzy',
 'PipelineExecutionDisplayName': 'execution-1709513382874',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2024, 3, 4, 0, 49, 42, 819000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 3, 4, 0, 49, 42, 819000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:645257835735:user-profile/d-mjncnq1fbvn8/cwatson',
  'UserProfileName': 'cwatson',
  'DomainId': 'd-mjncnq1fbvn8'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:645257835735:user-profile/d-mjncnq1fbvn8/cwatson',
  'UserProfileName': 'cwatson',
  'DomainId': 'd-mjncnq1fbvn8'},
 'ResponseMetadata': {'RequestId': 'a101f545-09bd-440b-9b80-be17ed29c6c9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a101f545-09bd

In [33]:
# wait for execution to end
execution.wait()

In [34]:
# show all pipeline steps
execution.list_steps()

[{'StepName': 'RegisterBesLGBTModel-RegisterModel',
  'StartTime': datetime.datetime(2024, 3, 4, 0, 58, 36, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 3, 4, 0, 58, 37, 390000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 1,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:645257835735:model-package/tuning-job-model-packages/4'}}},
 {'StepName': 'CheckMSEEvaluation',
  'StartTime': datetime.datetime(2024, 3, 4, 0, 58, 34, 999000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 3, 4, 0, 58, 35, 368000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 1,
  'Metadata': {'Condition': {'Outcome': 'True'}}},
 {'StepName': 'CreateBestModel-CreateModel',
  'StartTime': datetime.datetime(2024, 3, 4, 0, 53, 57, 68000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 3, 4, 0, 54, 1, 845000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 1,
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-e

In [35]:
from pprint import pprint

# print evaluation
evaluation_json = sagemaker.s3.S3Downloader.read_file(
    "{}/evaluation.json".format(
        step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
    )
)
pprint(json.loads(evaluation_json))

{'regression_metrics': {'mse': {'standard_deviation': 0.027075540902981504,
                                'value': 0.0007515934073515174}}}


In [36]:
# start the fail pipeline with a threshold that's too strict
execution = pipeline.start(parameters=dict(MseThreshold=0.00001))

In [37]:
# wait for failure
try:
    execution.wait()
except Exception as error:
    print(error)

Waiter PipelineExecutionComplete failed: Waiter encountered a terminal failure state: For expression "PipelineExecutionStatus" we matched expected path: "Failed"


In [38]:
# show steps
execution.list_steps()

[{'StepName': 'SmartGridMSEFail',
  'StartTime': datetime.datetime(2024, 3, 4, 1, 8, 41, 289000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 3, 4, 1, 8, 41, 643000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'AttemptCount': 1,
  'FailureReason': 'Execution failed due to MSE > 1e-05',
  'Metadata': {'Fail': {'ErrorMessage': 'Execution failed due to MSE > 1e-05'}}},
 {'StepName': 'CheckMSEEvaluation',
  'StartTime': datetime.datetime(2024, 3, 4, 1, 8, 40, 615000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 3, 4, 1, 8, 40, 863000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 1,
  'Metadata': {'Condition': {'Outcome': 'False'}}},
 {'StepName': 'EvaluateTopModel',
  'StartTime': datetime.datetime(2024, 3, 4, 1, 3, 12, 701000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 3, 4, 1, 8, 39, 884000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 1,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:64

In [39]:
from pprint import pprint

# print evaluation
evaluation_json = sagemaker.s3.S3Downloader.read_file(
    "{}/evaluation.json".format(
        step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
    )
)
pprint(json.loads(evaluation_json))

{'regression_metrics': {'mse': {'standard_deviation': 0.027896625709791012,
                                'value': 0.0007893907051686374}}}
